In [1]:
# Common imports
import os
from dotenv import load_dotenv
import json
import lolviz
import openai
import pandas as pd

# Import the key CrewAI classes
from crewai import Agent, Task, Crew

In [5]:
# Update on 13-09-2024 to use ipython magic %pip rather than !pip for greater compatibility with VS Code
%pip install python-dotenv --quiet

%pip install crewai --quiet
%pip install crewai-tools --quiet

%pip install tiktoken --quiet

%pip install lolviz --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached crewai-1.2.1-py3-none-any.whl (552 kB)
  Using cached chromadb-1.1.1-cp39-abi3-win_amd64.whl (19.8 MB)
  Using cached instructor-1.12.0-py3-none-any.whl (157 kB)
  Using cached mcp-1.19.0-py3-none-any.whl (170 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.38.0-py3-none-any.whl (19 kB)
  Using cached pdfplumber-0.11.7-py3-none-any.whl (60 kB)
  Using cached pyvis-0.3.2-py3-none-any.whl (756 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Using cached docstring_parser-0.17.0-py3-none-any.whl (36 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
  Using cached pre_commit-4.3.0-py2.py3-none-any.whl (220 kB)
  Using cached sse_starlette-3.0.2-py3-none-any.whl (11 kB)
  Using cached starlette-0.48.0-py3-none-any.whl (73 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
  Using cached pdfminer_six-20250506-py3-none-any.whl (5.6 MB)
  Using cached cryptography-46.0.3-cp311-


[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import tiktoken

In [3]:
load_dotenv('.env')

True

In [4]:
# Pass the API Key to the OpenAI Client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [5]:
# This is the "Updated" helper function for calling LLM
def get_completion(prompt, model="gpt-4o-mini", temperature=0, top_p=1.0, max_tokens=1024, n=1, json_output=False):
    if json_output == True:
      output_json_structure = {"type": "json_object"}
    else:
      output_json_structure = None

    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create( #originally was openai.chat.completions
        model=model,
        messages=messages,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        n=1,
        response_format=output_json_structure,
    )
    return response.choices[0].message.content

In [10]:
# Note that this function directly take in "messages" as the parameter.
def get_completion_from_messages(messages, model="gpt-4o-mini", temperature=0, top_p=1.0, max_tokens=1024, n=1):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        n=1
    )
    return response.choices[0].message.content

In [7]:
# This function is for calculating the tokens given the "message"
# ⚠️ This is simplified implementation that is good enough for a rough estimation

def count_tokens(text):
    encoding = tiktoken.encoding_for_model('gpt-4o-mini')
    return len(encoding.encode(text))


def count_tokens_from_message(messages):
    encoding = tiktoken.encoding_for_model('gpt-4o-mini')
    value = ' '.join([x.get('content') for x in messages])
    return len(encoding.encode(value))

In [ ]:
def identify_query_type(user_message):
    delimiter = "####"

    system_message = f"""
    You will be provided with customer service queries. \
    The customer service query will be enclosed in
    the pair of {delimiter}.

    Classify the customer service query into one of these categories below:
    1) "general_info" if the query is about school location, address, postal code, telephone number and nearby public transportation options
    2) "subjects_offered" if the query is about academic subjects offered by the school
    3) "cca" if the query is about co-curricular activities such as clubs, sports, performing arts etc.
    4) "unknown" if the query does not fit any category above.

    Ensure your response contains only one word: general_info, subjects_offered, cca or unknown.
    """

    messages =  [
        {'role':'system',
         'content': system_message},
        {'role':'user',
         'content': f"{delimiter}{user_message}{delimiter}"},
    ]
    type_of_query = get_completion_from_messages(messages)
    #type_of_query = type_of_query.replace("'", "\"")
    #type_of_query = json.loads(type_of_query)
    return type_of_query

In [ ]:
user_query = "I want to know what subjects are taught in Dunman High."
result = identify_query_type(user_query)
print(result) 

unknown


In [20]:
from langchain.agents import Tool
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
import pandas as pd

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from crewai import Agent, Task, Crew, Process


In [22]:
df_address = pd.read_csv(".\data\moe_generalinfo.csv")

In [24]:
df_cca = pd.read_csv(".\data\moe_cca.csv")

In [25]:
df_subjects = pd.read_csv(".\data\moe_subjects.csv")

In [27]:
df_cca.head(10)

,School_name,school_section,cca_grouping_desc,cca_generic_name,cca_customized_name
0,ADMIRALTY PRIMARY SCHOOL,PRIMARY,ART AND CRAFTS,CLUBS AND SOCIETIES,na
1,ADMIRALTY PRIMARY SCHOOL,PRIMARY,CHINESE DANCE,VISUAL AND PERFORMING ARTS,na
2,ADMIRALTY PRIMARY SCHOOL,PRIMARY,CHOIR,VISUAL AND PERFORMING ARTS,na
3,ADMIRALTY PRIMARY SCHOOL,PRIMARY,DESIGN AND INNOVATION,CLUBS AND SOCIETIES,na
4,ADMIRALTY PRIMARY SCHOOL,PRIMARY,"ENGLISH LANGUAGE, DRAMA AND DEBATING",CLUBS AND SOCIETIES,na
5,ADMIRALTY PRIMARY SCHOOL,PRIMARY,FOOTBALL,PHYSICAL SPORTS,na
6,ADMIRALTY PRIMARY SCHOOL,PRIMARY,GIRL GUIDES (BROWNIES),UNIFORMED GROUPS,na
7,ADMIRALTY PRIMARY SCHOOL,PRIMARY,GUITAR ENSEMBLE,VISUAL AND PERFORMING ARTS,na
8,ADMIRALTY PRIMARY SCHOOL,PRIMARY,GUZHENG ENSEMBLE,VISUAL AND PERFORMING ARTS,na
9,ADMIRALTY PRIMARY SCHOOL,PRIMARY,INDIAN DANCE,VISUAL AND PERFORMING ARTS,na


In [29]:
# create pandas tools for each dataset

from crewai.tools import tool

pandas_address_agent = create_pandas_dataframe_agent(
    llm=ChatOpenAI(temperature=0, model='gpt-4o-mini'),
    df=df_address, 
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True # <-- This is an "acknowledgement" that this can run potentially dangerous code
)

@tool('Pandas School Address Tool')
def pandas_address_tool(query: str) -> str:
    """
    Manipulate and analyse tabular data 
    """
    return pandas_address_agent.run(query)

pandas_subjects_agent = create_pandas_dataframe_agent(
    llm=ChatOpenAI(temperature=0, model='gpt-4o-mini'),
    df=df_subjects, 
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True # <-- This is an "acknowledgement" that this can run potentially dangerous code
)

@tool('Pandas School Subjects Tool')
def pandas_subjects_tool(query: str) -> str:
    """
    Manipulate and analyse tabular data 
    """
    return pandas_subjects_agent.run(query)

pandas_cca_agent = create_pandas_dataframe_agent(
    llm=ChatOpenAI(temperature=0, model='gpt-4o-mini'),
    df=df_cca, 
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True # <-- This is an "acknowledgement" that this can run potentially dangerous code
)

@tool('Pandas School Subjects Tool')
def pandas_cca_tool(query: str) -> str:
    """
    Manipulate and analyse tabular data 
    """
    return pandas_cca_agent.run(query)

In [39]:
# Creating Agents
agent_address_analyst = Agent(
    role="Data Analyst",
    goal="Retrieve the relevant data based on user query: {query}",
    backstory="""You're the best data retriever.""",
    allow_delegation=False,
	verbose=True,
    tools=[pandas_address_tool],
)

task_address = Task(
    description="""\
    1. Understand the user query: {query}.
    2. Use the tool to retrieve the data based on the user query. For example, extract the school name from {query} and use the school name to look up the relevant information.
    3. Present the retrieved information in a friendly and easy-to-understand manner.""",

    expected_output="""\
    Retrieved information about the relevant school in the user query.""",

    agent=agent_address_analyst,
)

agent_subjects_analyst = Agent(
    role="Data Analyst",
    goal="Retrieve the relevant data based on user query: {query}",
    backstory="""You're the best data retriever.""",
    allow_delegation=False,
	verbose=True,
    tools=[pandas_subjects_tool],
)

task_subjects = Task(
    description="""\
    1. Understand the user query: {query}.
    2. Use the tool to retrieve the data based on the user query. For example, extract the school name from {query} and use the school name to look up the relevant information.
    3. Present the retrieved information in a friendly and easy-to-understand manner.""",

    expected_output="""\
    Retrieved information about the relevant school in the user query.""",

    agent=agent_subjects_analyst,
)

agent_cca_analyst = Agent(
    role="Data Analyst",
    goal="Retrieve the data based on user query: {query}",
    backstory="""You're the best data retriever.""",
    allow_delegation=False,
	verbose=True,
    tools=[pandas_cca_tool],
)

task_cca = Task(
    description="""\
    1. Understand the user query: {query}.
    2. Use the tool to retrieve the data based on the user query. For example, extract the school name from {query} and use the school name to look up the relevant information.
    3. Present the retrieved information in a friendly and easy-to-understand manner.""",

    expected_output="""\
    Retrieved information about the relevant school in the user query.""",

    agent=agent_cca_analyst,
)

In [40]:
user_query = "I want to know what subjects are taught in Dunman High."
result = identify_query_type(user_query)
print(result) 

if result== "general_info":
    selected_agent = agent_address_analyst
elif result == "subjects_offered":
    selected_agent = agent_subjects_analyst
elif result == "cca":
    selected_agent = agent_cca_analyst
else:
    selected_agent = agent_address_analyst

if result== "general_info":
    selected_task = task_address
elif result == "subjects_offered":
    selected_task = task_subjects
elif result == "cca":
    selected_task = task_cca
else:
    selected_task = task_address


print(selected_agent)
print(selected_task)



subjects_offered
id=UUID('2c7dfb6d-b657-45b2-b9ac-8beb643baa32') role='Data Analyst' goal='Retrieve the relevant data based on user query: {query}' backstory="You're the best data retriever." config=None cache=True verbose=True max_rpm=None allow_delegation=False tools=[Tool(name='Pandas School Subjects Tool', description="Tool Name: Pandas School Subjects Tool\nTool Arguments: {'query': {'description': None, 'type': 'str'}}\nTool Description: \n    Manipulate and analyse tabular data \n    ", env_vars=[], args_schema=<class 'abc.Pandasschoolsubjectstool'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x0000028AB45E62A0>, result_as_answer=False, max_usage_count=None, current_usage_count=3, func=<function pandas_subjects_tool at 0x0000028ABA5E8E00>)] max_iter=25 agent_executor=<crewai.agents.crew_agent_executor.CrewAgentExecutor object at 0x0000028ABA7C26D0> llm=<crewai.llms.providers.openai.completion.OpenAICompletion object at 0x0000028ABA7C1890> crew=None 

In [41]:
# Create and run crew

crew = Crew(
    agents = [selected_agent],
    tasks = [selected_task],
    verbose = True,
    process = Process.sequential
)

In [42]:
output = crew.kickoff()
return output

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b36864f2-37e2-4707-bb63-38ba6979d1b0                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Analyst                                                                                            │
│                                                                                                                 │
│  Task:     1. Understand the user query: {query}.                                                               │
│      2. Use the tool to retrieve the data based on the user query. For example, extract the school name from    │
│  {query} and use the school name to look up the relevant information.                                           │
│      3. Present the retrieved information in a friendly and easy-to-understand manner.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\user\Desktop\submission1\venv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


c:\Users\user\Desktop\submission1\venv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation 
error for Pandasschoolsubjectstool
query
  Field required [type=missing, input_value={'description': "retrieve...8772', 'metadata': {}}}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing.
 Tool Pandas School Subjects Tool accepts these inputs: Tool Name: Pandas School Subjects Tool
Tool Arguments: {'query': {'description': None, 'type': 'str'}}
Tool Description: 
    Manipulate and analyse tabular data 
    
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Task...
    └── 🔧 Failed Pandas School Subjects Tool (5)[0m
🚀 Crew: crew
└── 📋 Task: f1dad852-4ab2-4b0a-8ccd-4622e2c35cb0
    Status: Executing Ta

...

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: b36864f2-37e2-4707-bb63-38ba6979d1b0                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  Tool Usage Failed                                                                                              │
│  Name: Pandas School Subjects Tool                                                                              │
│  Error: Arguments validation failed: 1 validation error for Pandasschoolsubjectstool                            │
│  query                                                                                                          │
│    Field required [type=missing, input_value={'description': "retrieve...8772', 'metadata': {}}}},              │
│  input_type=dict]                                                                                               │
│      For further information visit https://errors.pydantic.dev/2.12/v/missing                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

RecursionError: maximum recursion depth exceeded